In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume-entities-for-ner/Entity Recognition in Resumes.json


In [2]:
# import logging
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [3]:
data = trim_entity_spans(convert_dataturks_to_spacy("../input/resume-entities-for-ner/Entity Recognition in Resumes.json"))
data[0]

["Abhishek Jha Application Development Associate - Accenture  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company's growth in best possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK EXPERIENCE  Application Development Associate  Accenture -  November 2017 to Present  Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries for the Bot which will be triggered based on given input. Also, Training the bot for different possible utterances (Both positive and negative), which will be given as input by the user.  EDUCATION  B.E in Information science and engineering  B.v.b college of engineering and technology -  Hubli, Karnataka  August 2013 to June 2017  12th in Mathematics  Woodbine modern school  April 2011 to March 2013  10th  Kendriya Vidyalaya  April 2001 to March 2011  SKILLS  C (Le

In [4]:
!pip install spacy==2.1.4

     |████████████████████████████████| 29.8 MB 464 kB/s eta 0:00:011
     |████████████████████████████████| 54 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 3.2 MB 47.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 54.8 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 279 kB/s  eta 0:00:01
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.5
    Uninstalling preshed-3.0.5:
      Successfully uninstalled preshed-3.0.5
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: blis
    Found existing installation: blis 0.7.4
    Uninstalling blis-0.7.4:
      Successfully uninstalled blis-0.7.4
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5
  Attempting uninstall: jsonschema
    Found existing installati

### Train Test Split

In [7]:
import random
import math

def train_test_split(data, test_size, random_state):

    random.Random(random_state).shuffle(data)
    test_idx = len(data) - math.floor(test_size * len(data))
    train_set = data[0: test_idx]
    test_set = data[test_idx: ]

    return train_set, test_set

In [8]:
train_data, test_data = train_test_split(data, test_size = 0.1, random_state = 42)

In [9]:
import spacy

def train_spacy():
    
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
        
    # add labels
    for _, annotations in train_data:
         for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [10]:
nlp = train_spacy()

Statring iteration 0
{'ner': 25490.853286224534}
Statring iteration 1
{'ner': 18053.99950617645}
Statring iteration 2
{'ner': 19306.84766413166}
Statring iteration 3
{'ner': 13870.722479097922}
Statring iteration 4
{'ner': 13669.987254515387}
Statring iteration 5
{'ner': 10813.1269784428}
Statring iteration 6
{'ner': 9642.527375285339}
Statring iteration 7
{'ner': 9706.868706010742}
Statring iteration 8
{'ner': 7484.900262942686}
Statring iteration 9
{'ner': 8713.432655944167}


In [11]:
nlp.to_disk('nlp_model')

In [19]:
nlp_model = spacy.load('/kaggle/working/nlp_model')

In [20]:
nlp_model

In [21]:
print(nlp_model(train_data[0][0]))

Shrishti Chauhan Have total work experience of 2.5 years on Oracle Fusion Middleware - SOA, WebLogic and MFT Module.  Bilaspur, Chhattisgarh - Email me on Indeed: indeed.com/r/Shrishti- Chauhan/89d7feb4b3957524  Seeking to hone and enhance my technical skills in Oracle Fusion Middleware while working as a professional in challenging and goal oriented environment.  Willing to relocate to: Bengaluru, Karnataka  WORK EXPERIENCE  Technical Consultant  Oracle -  Bengaluru, Karnataka -  October 2015 to Present  • Have total work experience of 2.5 years on Oracle Fusion Middleware - SOA, WebLogic and MFT Module.  • Have extensively worked on Support, Testing, Cloning, Monitoring and Maintenance support and Enhancement for the E-Commerce Project with multi system module.  • Have good understanding on End-to-End Business Process.  • Experience in developing and deploying BPEL Processes using technology adapters (DB Adapter, File Adapter, FTP Adapter and JMS Adapter).  • Part of a team for devel

In [24]:
doc = nlp_model(train_data[0][0])

for ent in doc.ents:
#     print(ent)
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Shrishti Chauhan
YEARS OF EXPERIENCE           - 2.5 years
COMPANIES WORKED AT           - Oracle
COMPANIES WORKED AT           - Oracle
DESIGNATION                   - Technical Consultant
COMPANIES WORKED AT           - Oracle
LOCATION                      - Bengaluru
COMPANIES WORKED AT           - Oracle
COMPANIES WORKED AT           - Oracle
COMPANIES WORKED AT           - Oracle
COMPANIES WORKED AT           - Oracle
DESIGNATION                   - Technical Consultant
COMPANIES WORKED AT           - Oracle
YEARS OF EXPERIENCE           - 2.5 years
COMPANIES WORKED AT           - Oracle
DEGREE                        - C.S.
COLLEGE NAME                  - CHHATTISGARH SWAMI VIVEKANANDA TECHNICAL UNIVERSITY
SKILLS                        - Xml, Oracle MFT, Core Java, Oracle SOA, WSDL, ODI
SKILLS                        - ERP Packages Oracle Fusion Middleware and Oracle Fusion Application (DOO/GOP) Fusion Middleware Modules Oracle SOA, MFT and Web Logic